# CNN Models

Created: 2019.10.21  
Notebook sequence: 9

For experimenting with keras CNN models

---

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import util
import pandas as pd
import keras
import numpy as np
from tqdm import tqdm_notebook as tqdm
from keras.layers import Dense, Conv2D, Flatten, Conv1D, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
#data = util.load_dataset("selection_reliability__13_w2v/sequence.pkl.pkl")
#df = util.load_selection_dataset("selection_reliability_os_13")

In [5]:
df

,date,source,name,content,reliable
0,2018-11-12,oann,Germanys Scholz sees Italy taking steps to res...,BERLIN (Reuters) German Finance Minster Olaf ...,1
1,2018-04-29,oann,Rivals see ruse in Palestinian political forum...,GAZA (Reuters) Political rivals accused Pales...,1
2,2018-10-08,oann,Venezuela says jailed lawmaker commits suicide...,CARACAS (Reuters) Venezuelas government said ...,1
3,2018-08-29,oann,Oil edges up on looming Iran sanctions but ris...,SINGAPORE (Reuters) Oil markets were stable o...,1
4,2018-11-03,oann,US allows eight importers to keep buying Iran ...,WASHINGTON (Reuters) The United States said o...,1
...,...,...,...,...,...
10651,2018-08-15,Veterans Today,Radioactive sheep boost claims of secret Israe...,"On September 22, 1979, US satellite Vela 6911 ...",0
10652,2018-05-09,Veterans Today,Obama Dear Trump youre an idiot - Veterans Tod...,Obama once threw the gauntlet down and declare...,0
10653,2018-08-06,Veterans Today,WashPost Trump promised to fix veterans proble...,President Donald Trump told the countrys 20 mi...,0
10654,2018-09-11,Veterans Today,911 Israel One America Zero,"Kevin sent me a photo still, which I traced to...",0


In [15]:
df.reliable.value_counts().index.tolist()

[1, 0]

In [4]:
y = np.array(list(df.reliable))
X = np.array(data)
X_padded = keras.preprocessing.sequence.pad_sequences(X, maxlen=500, padding='post', dtype='float32')
# X = np.expand_dims(X, axis=2)
print(X_padded.shape)
print(y.shape)

(30000, 500, 300)
(30000,)


In [5]:
X_padded = np.reshape(X_padded, (X_padded.shape[0], 500*300, 1))
X_padded.shape

(30000, 150000, 1)

In [9]:
model = keras.Sequential()
#model.add(Flatten(input_shape=(500, 300, 1)))
model.add(Conv1D(16, kernel_size=300, strides=300, activation='relu', input_shape=(300*500, 1)))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(Dropout(.2))
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

adam_optimizer = keras.optimizers.Adam(learning_rate=.001, decay=1e-6)
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()
model.fit(X_padded, y, batch_size=128, validation_split=.2, epochs=100, verbose=1, use_multiprocessing=True, workers=4)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 500, 16)           4816      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 498, 16)           784       
_________________________________________________________________
dropout_1 (Dropout)          (None, 498, 16)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7968)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                255008    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 260,641
Trainable params: 260,641
Non-trainable params: 0
________________________________________________

KeyboardInterrupt: 